In [16]:
import geopandas as gpd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
load_dotenv()
#Write the data to PostGIS
db_connection_url = os.getenv('READ_DB_URL')

# Create a SQLAlchemy engine
engine = create_engine(db_connection_url)
# Write the GeoDataFrame to PostGIS
sql = "SELECT * from resort where skimap_id is not null"
df = gpd.GeoDataFrame.from_postgis(sql, engine)  


In [17]:
list_of_dicts = df.to_dict('records')

In [18]:
list_of_tuples = []
for d in list_of_dicts:
    tup = (d["passaffiliation"],d["partnered"],d["skimap_id"])
    list_of_tuples.append(tup)

In [19]:
import psycopg2

# Database connection parameters
DB_PARAMS = {
    "dbname": os.getenv('PGDATABASE'),
    "user": os.getenv('PGUSER'),
    "password": os.getenv('PGPASSWORD'),
    "host": os.getenv('PGHOST'),
    "port": "5432"
}

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()

    # Bulk update query for two columns
    update_query = """
    UPDATE ski_resorts_skiarea
    SET passaffiliation = %s, partnered = %s
    WHERE id = %s
    """
    #Use list of tuples constructed above to update, tuple should be ordered in the order that the %s appears so (passaffiliation,partnered,id)


    # Execute the bulk update
    cursor.executemany(update_query, list_of_tuples)

    # Commit the transaction
    conn.commit()

    print("Bulk update completed successfully.")

except psycopg2.Error as e:
    print("Error updating records:", e)

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Bulk update completed successfully.
